In [70]:
import numpy as np

X = np.array([[0.84147,-0.95892, 0.41212, 0.42017,-0.96140], [0.90930,-0.27942,-0.54402, 0.99061,-0.75099], [0.14112, 0.65699,-0.99999, 0.65029, 0.14988], [-0.75680, 0.98936,-0.53657,-0.28790, 0.91295]])
pval = np.array([0.8414710 ,0.1411200 ,0.9589243 ,0.6569866 ,0.4121185 ,0.9999902 ,0.4201670 ,0.6502878 ,0.9613975 ,0.1498772 ,0.8366556 ,0.8462204 ,0.0010000 ,1.0000000])
yval = np.array([1 ,0 ,0 ,1 ,0 ,0 ,0 ,1 ,0 ,0 ,1 ,0 ,1 ,0])
X_rec = np.array([[0.84147,-0.95892,0.41212,0.42017,-0.96140],[0.90930,-0.27942,-0.54402,0.99061,-0.75099],[0.14112,0.65699,-0.99999,0.65029,0.14988],[-0.75680,0.98936,-0.53657,-0.28790,0.91295]])
Theta = np.array([[0.5403023,-0.6536436,0.7539023,-0.8390715,0.9074468],[-0.4161468,0.2836622,-0.1455000,0.0044257,0.1367372],[-0.9899925,0.9601703,-0.9111303,0.8438540,-0.7596879]])
num_users = 3
num_movies = 4
num_features = 5
lambda_ = 0
R = np.array([[1,0,0],[0,0,0],[0,0,1],[1,1,0]])
Y = np.array([[0.84147,0.00000,-0.00000],[0.00000,-0.00000,0.00000],[-0.00000,0.00000,0.83666],[0.65699,0.65029,-0.00000]])

In [48]:
def estimateGaussian(X):
    m, n = X.shape
    mu = np.zeros((n, 1))
    sigma2 = np.zeros((n, 1))
    mu = (1/m)* np.sum(X,axis=0)
    for i in range(m):
        sigma2[i] = (1/m)*np.transpose(X[:,i] - mu[i]).dot(X[:,i] - mu[i])
    return mu,sigma2.flatten()

In [63]:
def selectThreshold(yval, pval):
    bestEpsilon = 0
    bestF1 = 0
    F1 = 0
    stepsize = (max(pval) - min(pval)) / 1000
    m = yval.shape[0]
    epsilon = min(pval)
    while epsilon <= max(pval):
        epsilon = epsilon + stepsize
        y_pred = np.ones((m,1))
        y_pred = pval < epsilon
        tp = np.sum((y_pred == 1) & (yval == 1))
        tn = np.sum((y_pred == 0) & (yval == 0))
        fp = np.sum((y_pred == 1) & (yval == 0))
        fn = np.sum((y_pred == 0) & (yval == 1))
        prec = tp / (tp+fp)
        rec= tp/(tp+fn)
        F1 = (2*prec*rec)/(prec + rec)
        if F1 > bestF1:
             bestF1 = F1
             bestEpsilon = epsilon
    return bestEpsilon,bestF1

In [97]:
def cofiCostFunc(X, Theta, Y, R, num_users, num_movies, num_features, lambda_):
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)
    h_theta_temp = np.multiply(X.dot(np.transpose(Theta)),R)
    J = ((1/2)*np.transpose((h_theta_temp - Y).flatten()).dot((h_theta_temp - Y).flatten()) + 
    (lambda_/2)* np.transpose(Theta.flatten()).dot(Theta.flatten()) + 
    (lambda_/2)* np.transpose(X.flatten()).dot(X.flatten())) 
    Theta_grad = np.transpose(h_theta_temp - Y).dot(X) + lambda_*(Theta)
    X_grad = (h_theta_temp - Y).dot(Theta) + lambda_*X
    return J,X_grad,Theta_grad